In [1]:
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"
from nibabel.testing import data_path
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import time
import datetime
import json

In [2]:
import albumentations as A

In [3]:
#### pip install numpy==1.18.5

In [4]:
from dltk.io.augmentation import *
from dltk.io.preprocessing import *
from scipy.ndimage.filters import gaussian_filter
import SimpleITK as sitk

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-09-15 07:25:12.857915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30967 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [6]:
file_list_train = glob(os.path.join("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_train", "*"))
file_list_test = glob(os.path.join("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_test", "*"))

X = np.array(np.load(file_list_train[0])['X_train'])
Y = np.array(np.load(file_list_train[0])['Y_train'])

In [7]:
IMG_HEIGHT = X.shape[1]
IMG_WIDTH  = X.shape[2]
IMG_CHANNELS = X.shape[3]

In [8]:
def load_image_train(image_file):
    data = np.load(image_file)
    return data['X_train'], data['Y_train']

def load_image_test(image_file):
    data = np.load(image_file)
    return data['X_test'], data['Y_test']

In [9]:
def resize(input_image, real_image, height, width):
    shape = tf.shape(input_image)
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image

In [10]:
def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(
        stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 4])

    return cropped_image[0], cropped_image[1]

In [11]:
def random_translation(X,Y):
    i = tf.random.uniform(shape = [], minval=-15, maxval=15, dtype=tf.dtypes.int32)
    j = tf.random.uniform(shape = [], minval=-15, maxval=15, dtype=tf.dtypes.int32)
    Y = tf.roll(Y, [i,j], axis = [0,1])
    X = tf.roll(X, [i,j], axis = [0,1])
    return X,Y
    

In [12]:
transforms = A.Compose([
        #A.HorizontalFlip(p=0.5),
        #A.RandomRotate90(p=0.5),
        #A.VerticalFlip(p=0.5),
        #A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.6, p = 0.2),
        A.ElasticTransform(p=0.2, alpha=200, sigma=200 * 0.06, alpha_affine=200 * 0.02),
        #A.GaussianBlur(p = 0.2, blur_limit=(3, 7), sigma_limit=0),
        #A.GaussNoise (p=0.2, var_limit=(0.2, 0.8), mean=0., per_channel=True, always_apply=False),
        #A.OpticalDistortion (p = 0.5, distort_limit=0.05, shift_limit=0.05, interpolation=1, border_mode=4),
        #A.PixelDropout (dropout_prob=0.02, per_channel=False, drop_value=-1, p=0.2),
        #A.ZoomBlur (max_factor=1.11, step_factor=(0.01, 0.06), always_apply=False, p=0.5)
    ])



def aug_fn(image, mask):
    aug = transforms(image = image, mask = mask)
    aug_img, aug_mask = aug["image"], aug["mask"]

    return aug_img, aug_mask

In [13]:
def process_data(image, mask):
    image = tf.cast(image, tf.float32)
    mask = tf.cast(mask, tf.float32)
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, mask], Tout=[tf.float32, tf.float32])
    return aug_img[0], aug_img[1]

In [14]:
@tf.function()
def random_jitter(input_image, real_image):
    input_image = tf.cast(input_image, tf.double)
    input_image = tf.reshape(input_image, [192,192,4])
    real_image = tf.reshape(real_image, [192,192,4])
    
    #if tf.random.uniform(())> 0.5:
    #    input_image, real_image = random_translation(input_image, real_image)
        
    input_image, real_image = resize(input_image, real_image, 250, 250)

    input_image, real_image = random_crop(input_image, real_image)

    if tf.random.uniform(()) > 0.5:
        # Random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)
        
    #input_image, real_image = process_data(input_image, real_image)

    return input_image, real_image

def test_reshape(X,Y):
    X = tf.cast(X, tf.double)
    X = tf.reshape(X, [192,192,4])
    Y = tf.reshape(Y, [192,192,4])
    return X,Y
    

In [15]:
BATCH_SIZE = 10
BUFFER_SIZE = 500

In [16]:
train_dataset = tf.data.Dataset.list_files('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_train3/*.npz')
train_dataset = train_dataset.map(lambda item: tf.numpy_function(
          load_image_train, [item], [tf.double, tf.double]),
          num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.map(random_jitter, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
#train_dataset = train_dataset.map(process_data, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

2023-09-15 07:25:20.110474: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30967 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [17]:
test_dataset = tf.data.Dataset.list_files('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_test/*.npz')
test_dataset = test_dataset.map(lambda item: tf.numpy_function(
          load_image_test, [item], [tf.double, tf.double]),
          num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(test_reshape)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [18]:
def conv_block(input, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(input)
    x = tf.keras.layers.BatchNormalization()(x)   #Not in the original network. 
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)  #Not in the original network
    x = tf.keras.layers.Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = tf.keras.layers.MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = tf.keras.layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = tf.keras.layers.Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = tf.keras.layers.Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = tf.keras.Model(inputs, outputs, name="U-Net")
    return model

In [19]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
input_shape

(192, 192, 4)

In [20]:
unet = build_unet(input_shape,Y.shape[-1])
tf.keras.utils.plot_model(unet, show_shapes=True, dpi=64);

softmax
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [21]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0,:,:,1], np.argmax(tar[0], axis =2), np.argmax(prediction[0], axis = 2)]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [22]:
#unet_optimizer = tf.keras.optimizers.SGD(learning_rate=2e-4, momentum=0.0)
unet_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5, epsilon = 1e-4)

In [23]:
from IPython import display

In [24]:
def dice0(y_true, y_pred, smooth = 1e-7):
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,0], 'float32'), [-1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,0], 'float32'), [-1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f)))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2)+smooth)

def dice1(y_true, y_pred, smooth = 1e-7):  
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,1], 'float32'), [-1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,1], 'float32'), [-1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f)))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2)+smooth)

def dice2(y_true, y_pred, smooth = 1e-7):
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,2], 'float32'), [-1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,2], 'float32'), [-1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f)))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2)+smooth)

def dice3(y_true, y_pred, smooth = 1e-7):  
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,3], 'float32'), [-1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,3], 'float32'), [-1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f)))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2)+smooth)

def cce(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

def dice_loss1(y_true, y_pred):
    a0 = 0
    a1 = 1
    a2 = 1
    a3 = 1
    return 1-(a0*dice0(y_true,y_pred)+a1*dice1(y_true,y_pred)+a2*dice2(
        y_true,y_pred)+a3*dice3(y_true,y_pred))/(a0+a1+a2+a3)


In [25]:
os.mkdir('./Crossvalidation/Train3/model2')

In [26]:
epochs = 30
checkpoint_dir = './Crossvalidation/Train3/model2'
checkpoint_filepath = os.path.join(checkpoint_dir, "ckpt_Unet_{epoch:02d}")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose = 1,
    save_weights_only=False,
    monitor="val_loss",
    mode='min',
    save_best_only=False)

In [27]:
unet.compile(optimizer=unet_optimizer, loss=dice_loss1, metrics=[dice0,dice1,dice2,dice3, cce])

In [ ]:
history = unet.fit(train_dataset,
                    verbose=1, 
                    epochs=epochs,
                    #steps_per_epoch =X_train.shape[0]//BATCH_SIZE-1,
                    #validation_steps = X.shape[0]//BATCH_SIZE-1,
                    validation_data=test_dataset, 
                    callbacks=[model_checkpoint_callback],
                    shuffle=True)

Epoch 1/30


2023-09-15 07:25:38.686311: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-09-15 07:25:42.437849: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8300


187/187 [==============================] - 45s 171ms/step - loss: 0.5668 - dice0: 0.6076 - dice1: 0.0232 - dice2: 0.6890 - dice3: 0.5875 - cce: 2.0303 - val_loss: 0.7483 - val_dice0: 0.8280 - val_dice1: 0.0159 - val_dice2: 0.6169 - val_dice3: 0.1222 - val_cce: 0.8603

Epoch 00001: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_01


2023-09-15 07:26:25.034598: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Crossvalidation/Train3/model2/ckpt_Unet_01/assets
Epoch 2/30
187/187 [==============================] - 28s 139ms/step - loss: 0.4405 - dice0: 0.6058 - dice1: 0.0249 - dice2: 0.8237 - dice3: 0.8298 - cce: 2.3717 - val_loss: 0.5014 - val_dice0: 0.8202 - val_dice1: 0.0210 - val_dice2: 0.7033 - val_dice3: 0.7717 - val_cce: 1.1215

Epoch 00002: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_02
INFO:tensorflow:Assets written to: ./Crossvalidation/Train3/model2/ckpt_Unet_02/assets
Epoch 3/30
187/187 [==============================] - 29s 145ms/step - loss: 0.4288 - dice0: 0.6629 - dice1: 0.0297 - dice2: 0.8483 - dice3: 0.8355 - cce: 2.2409 - val_loss: 0.4898 - val_dice0: 0.8599 - val_dice1: 0.0293 - val_dice2: 0.8127 - val_dice3: 0.6887 - val_cce: 1.0998

Epoch 00003: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_03
INFO:tensorflow:Assets written to: ./Crossvalidation/Train3/model2/ckpt_Unet_03/assets
Epoch 4/30
187/187 [===========


Epoch 00020: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_20
INFO:tensorflow:Assets written to: ./Crossvalidation/Train3/model2/ckpt_Unet_20/assets
Epoch 21/30
187/187 [==============================] - 42s 212ms/step - loss: 0.1016 - dice0: 0.9971 - dice1: 0.8687 - dice2: 0.9115 - dice3: 0.9149 - cce: 0.0308 - val_loss: 0.2346 - val_dice0: 0.9959 - val_dice1: 0.6793 - val_dice2: 0.8245 - val_dice3: 0.7923 - val_cce: 0.0482

Epoch 00021: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_21
INFO:tensorflow:Assets written to: ./Crossvalidation/Train3/model2/ckpt_Unet_21/assets
Epoch 22/30
187/187 [==============================] - 41s 209ms/step - loss: 0.0960 - dice0: 0.9972 - dice1: 0.8731 - dice2: 0.9173 - dice3: 0.9215 - cce: 0.0290 - val_loss: 0.2366 - val_dice0: 0.9952 - val_dice1: 0.6866 - val_dice2: 0.8167 - val_dice3: 0.7868 - val_cce: 0.0550

Epoch 00022: saving model to ./Crossvalidation/Train3/model2/ckpt_Unet_22
INFO:tensorflow:Assets written to: ./Cros

In [ ]:
unet.save('./Modelli/Unet_Allexp_60_120slice_50epch_Alldice_DataCgan_09_02_2022.hdf5')

In [ ]:
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train/History6")

In [ ]:
with open('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train/History6/unet_history.json', 'w') as handle: # saving the history of the model
    json.dump(history.history, handle)

In [ ]:
path = '/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train_nodef/History2/unet_history.json'

In [ ]:
history = json.load(open(path, 'r'))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
xposition = [27,28,29]
for xc in xposition:
    plt.axvline(x=xc, color='orange', linestyle='--')
    if xc == len(xposition): plt.axvline(x=xc, color='orange', linestyle='--', label = "ckpt")
    else: plt.axvline(x=xc, color='orange', linestyle='--')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xlim([0.8,30.5])
plt.ylim([0,0.5])
plt.legend(loc = "upper right")
plt.legend()
#plt.savefig("Images/Unet_history_finalckpt")
plt.show()

In [ ]:
loss = history['loss']
val_loss = history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xlim([0,30.5])
plt.ylim([0,1.4])
plt.legend(loc = "upper right")
plt.legend()
plt.show()

rec = history['dice0']
val_rec = history['val_dice0']

plt.plot(epochs, rec, 'y', label='Training Dice 0 ')
plt.plot(epochs, val_rec, 'r', label='Validation Dice 0')
plt.title('Training and Validation Dice 0')
plt.xlabel('Epochs')
plt.ylabel('Dice0')
plt.legend()
#plt.savefig("Images/Unet_Dice0")
plt.show()

rec1 = history['dice1']
val_rec1 = history['val_dice1']

plt.plot(epochs, rec1, 'y', label='Training Dice 1 ')
plt.plot(epochs, val_rec1, 'r', label='Validation Dice 1')
plt.title('Training and Validation Dice 1')
plt.xlabel('Epochs')
plt.ylabel('Dice1')
plt.xlim([0,30.5])
plt.ylim([0,1.0])
plt.legend()
#plt.savefig("Images/Unet_Dice1")
plt.show()

rec2 = history['dice2']
val_rec2 = history['val_dice2']


plt.plot(epochs, rec2, 'y', label='Training Dice 2')
plt.plot(epochs, val_rec2, 'r', label='Validation Dice 2')

plt.title('Training and Validation Dice 2')
plt.xlabel('Epochs')
plt.ylabel('Dice2')
plt.xlim([0,30.5])
plt.ylim([0,1.0])
plt.legend()
#plt.savefig("Images/Unet_Dice2")
plt.show()


rec3 = history['dice3']
val_rec3 = history['val_dice3']

l = [10,18,26,31]
plt.plot(epochs, rec3, 'y', label='Training Dice 3')
plt.plot(epochs, val_rec3, 'r', label='Validation Dice 3')
for i in l:
    plt.axvline(x=i, color='red', linestyle='--')

plt.title('Training and Validation Dice 3')
plt.xlabel('Epochs')
plt.ylabel('Dice3')
plt.xlim([0,30.5])
plt.ylim([0,1.0])
plt.legend()
#plt.savefig("Images/Unet_Dice3")
plt.show()

In [ ]:

model = keras.models.load_model("Modelli/Unet_Allexp_60_120slice_50epch_Alldice_DataCgan_09_02_2022.hdf5",
                                custom_objects={'dice0': dice0, 'dice1': dice1, 'dice2': dice2, 'dice3': dice3, "dice_loss1":dice_loss1})



In [ ]:
model.summary()

In [ ]:
for i,j in test_dataset:
    y = np.argmax(unet.predict(i), axis = 3)
    a = np.argmax(j, axis = 3)
    for k in range(y.shape[0]):
        plt.figure(figsize=(12, 8))
        plt.subplot(231)
        plt.title("Channel 0")
        plt.imshow(i[k,:,:,0])
        plt.subplot(232)
        plt.title("Predicted Seg")
        plt.imshow(y[k])
        plt.subplot(233)
        plt.title("Ground Truth ")
        plt.imshow(a[k])
        plt.show()

In [ ]:
for i,j in test_dataset:
    y = np.argmax(unet.predict(i), axis = 3)
    a = np.argmax(j, axis = 3)
    a = np.ma.masked_where(a == 0, a)

    for k in range(y.shape[0]):
        plt.figure(figsize=(5, 5))
        plt.title("BraTs segmentation")
        plt.imshow(i[k,:,:,0], cmap = "gist_gray")
        #plt.title("Predicted Seg")
        #plt.imshow(y[k], alpha = 0.1)
        #plt.title("Ground Truth ")
        plt.imshow(a[k], alpha = 0.5)
        plt.show()